This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "example"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "term_spread"

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [5]:
ts_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "allmonth.xls", usecols=["Date", "Spread"]).dropna().rename(columns={"Spread": "10Y-3M Term Spread"})

ts_df['Date']=ts_df['Date'].dt.to_period('M').dt.to_timestamp()

ts_df.set_index('Date', inplace=True)

today = dt.date.today()

# term_premium = fred.get_series(
#     series_id="THREEFYTP10",
#     observation_start=dt.date(2024, 1, 1),
#     observation_end=today,
#     frequency='m',
# ).rename("Term Premium on 10Y")


In [6]:
joined_df = pd.concat(
    [ts_df],#,, term_premium], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df = joined_df[joined_df.index >= '2023-06-01']


joined_df

,10Y-3M Term Spread
Date,
2023-06-01,-1.550807
2023-07-01,-1.494506
2023-08-01,-1.276580
2023-09-01,-1.087413
2023-10-01,-0.688249
2023-11-01,-0.915334
2023-12-01,-1.364093
2024-01-01,-1.303268
2024-02-01,-1.174093


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [10]:
# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tag <sup> for a subtitle 
fig.update_layout(
    title = dict(
        text = 'Term Spread<br><sup>Monthly</sup>',  # Add comma here
        font=dict(size=28)),
    xaxis_title=None,
    yaxis_title=None,
    legend=dict(
        font=dict(size=18)  # Change legend font size
    )
)

# This is graph specific, but here we want the y-axis to be billions of dollars
fig.update_yaxes(
    tickformat=",.1f",
    ticksuffix="%",
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
)

ut.add_end_labels(fig)

fig.update_layout(
    height=450,
    width=700,
)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")